<a href="https://colab.research.google.com/github/EmilyCarroll-del/Michael-J-Fox-Foundation-FOG-in-PD/blob/main/MJF_Additional_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#FAST FOURIER TRANSFORM

import numpy as np

N = 50  # Number of frequency components to keep

# Function for FFT filtering (keeping only low-frequency components)
def fft_filter(signal, N):
    """Perform FFT on a single signal and zero out high-frequency components."""
    # Perform FFT
    signal_fft = np.fft.fft(signal)

    # Zero out high-frequency components (keeping only the first N components)
    signal_fft[N:-N] = 0

    # Inverse FFT to get the filtered signal
    filtered_signal = np.fft.ifft(signal_fft).real

    return filtered_signal

# Apply FFT filtering to the accelerometer columns without downsampling
filtered_AccV = fft_filter(tdcsfog['AccV'], N)
filtered_AccML = fft_filter(tdcsfog['AccML'], N)
filtered_AccAP = fft_filter(tdcsfog['AccAP'], N)

# Assign filtered values to the new DataFrame columns
tdcsfog['AccV_filtered'] = filtered_AccV
tdcsfog['AccML_filtered'] = filtered_AccML
tdcsfog['AccAP_filtered'] = filtered_AccAP

# Display the updated DataFrame with filtered accelerometer data
print(tdcsfog.head())
print(tdcsfog.shape)

In [ ]:
# Apply FFT downsampling and time-domain downsampling to the accelerometer columns
filtered_AccV = fft_filter(defog['AccV'], N)
filtered_AccML = fft_filter(defog['AccML'], N)
filtered_AccAP = fft_filter(defog['AccAP'], N)

# Adjust the DataFrame index to match the downsampled signal length
# Keep only the first rows of the DataFrame that correspond to the downsampled signal length
defog = defog.iloc[:len(filtered_AccV)].copy()

# Assign downsampled values to the new DataFrame columns
defog['AccV_filtered'] = filtered_AccV
defog['AccML_filtered'] = filtered_AccML
defog['AccAP_filtered'] = filtered_AccAP

# Display the updated DataFrame with downsampled accelerometer data
print(defog.head())
print(defog.shape)

In [ ]:
#graphing max frequency of fourier transform
fs = 128
AccML_fft = np.fft.fft(tdcsfog['AccML'])
n = len(tdcsfog['AccML'])  # Number of samples
frequencies = np.fft.fftfreq(n, d=1/fs)
plt.figure(figsize=(10, 6))

# Only plot the positive frequencies (real-world frequencies)
positive_freqs = frequencies[:n//2]
magnitude = np.abs(AccML_fft[:n//2])  # Magnitude of the Fourier coefficients

plt.plot(positive_freqs, magnitude, label='FFT Magnitude')
plt.title('Fourier Transform of Accelerometer Data (AccML)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#SPECTROGRAM

from scipy.signal import spectrogram

# Parameters
window_size = 384
sampling_rate = 128

def compute_spectrogram(data, fs=sampling_rate, nperseg=window_size):
    # Compute the spectrogram
    frequencies, times, Sxx = spectrogram(data, fs=fs, window='hann', nperseg=nperseg)
    return Sxx  # Sxx is the 2D spectrogram matrix (frequency vs time)

# Define the number of samples per window
num_samples_per_window = window_size

# Lists to store spectrograms for each axis
spectrograms_v = []
spectrograms_ml = []
spectrograms_ap = []

# Loop through the DataFrame in chunks of `num_samples_per_window`
for start in range(0, len(tdcsfog) - num_samples_per_window + 1, num_samples_per_window):
    # Extract the window for each axis
    window_v = tdcsfog['AccV'].iloc[start:start + num_samples_per_window].values
    window_ml = tdcsfog['AccML'].iloc[start:start + num_samples_per_window].values
    window_ap = tdcsfog['AccAP'].iloc[start:start + num_samples_per_window].values

    # Compute the spectrogram for each axis
    S_v = compute_spectrogram(window_v)
    S_ml = compute_spectrogram(window_ml)
    S_ap = compute_spectrogram(window_ap)

    # Flatten each spectrogram to a 1D array if needed (for ML model input)
    spectrograms_v.append(S_v.flatten())
    spectrograms_ml.append(S_ml.flatten())
    spectrograms_ap.append(S_ap.flatten())

# Create DataFrames for each axis's spectrograms
spectrogram_v_df = pd.DataFrame(spectrograms_v)
spectrogram_ml_df = pd.DataFrame(spectrograms_ml)
spectrogram_ap_df = pd.DataFrame(spectrograms_ap)

# Combine into a single DataFrame with separate columns for each axis
# (Optional: you could concatenate these if your model can take multiple channels)
spectrogram_df = pd.concat([spectrogram_v_df, spectrogram_ml_df, spectrogram_ap_df], axis=1)

# Inspect the final DataFrame
print(spectrogram_df.head())

#print(spectrogram_v_df.head())
print(spectrogram_df.shape)